In [ ]:
#pip install keras_resnet

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import Input, Model
import tensorflow.keras as keras
import tensorflow as tf
from keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import History, ModelCheckpoint, TensorBoard
import numpy as np
#import keras_resnet.blocks
#import tcn
#from tcn import TCN, tcn_full_summary
import pickle
from urllib.request import urlopen
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
#from tcn import compiled_tcn
import os
#import keras_resnet.models

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#with open('/content/drive/My Drive/wavenet-test/features.pickle', 'rb') as handle:
#    features = pickle.load(handle)
    
#with open('/content/drive/My Drive/wavenet-test/labels.pickle', 'rb') as handle:
#  labels = pickle.load(handle)

#!cp '/content/drive/My Drive/wavenet-test/US8K.tar' .

In [ ]:
!nvidia-smi

Sat Dec  5 20:15:26 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
## Utils

def create_new_model():
  input_shape = (334, 224, 3)

  base_model = keras.applications.ResNet50(weights='imagenet',
                                           include_top=False,
                                           input_shape=input_shape)
  # adding regularization
  #regularizer = tf.keras.regularizers.l2(0.001)
  #for layer in base_model.layers:
  #  for attr in ['kernel_regularizer']:
  #      if hasattr(layer, attr):
  #        setattr(layer, attr, regularizer)
  #print(base_model.losses)
  #base_model.save_weights('resnet50.h5', save_format='h5')
  #base_model = tf.keras.models.model_from_json(base_model.to_json())
  # new_model would have random weights
  #base_model.load_weights('resnet50.h5')
  #print(base_model.losses)
  base_model.trainable = False

 
  inputs = keras.Input(shape=input_shape)
  # We make sure that the base_model is running in inference mode here,
  # by passing `training=False`. This is important for fine-tuning, as you will
  # learn in a few paragraphs.
  x = base_model(inputs, training=False)
  # Convert features of shape `base_model.output_shape[1:]` to vectors
  x = keras.layers.GlobalAveragePooling2D()(x)
  # A Dense classifier with a 10 units 
  outputs = keras.layers.Dense(10)(x)
  model = keras.Model(inputs, outputs)

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
  return model


def load_model(depth, filters, kernel_size, save_dir="./"):
  model = create_new_model(depth, filters, kernel_size)
  saved_model = save_dir + "saved_wavenet_clasifier.h5"
  # model.load here
  if os.path.isfile(saved_model):
    model.load_weights(saved_model)
  else:
    print("NO MODEL ON DISK, USING BRAND NEW ONE")
  return model

def create_callbacks(save_dir="./"):
  saved_model = save_dir + "saved_wavenet_clasifier.h5"
  saved_hist  = save_dir + 'wavenet_classifier_training_history.csv'
  checkpointer = ModelCheckpoint(filepath=saved_model, monitor='val_accuracy', verbose=1, save_best_only=True)
  #tensorboard  = TensorBoard(log_dir=save_dir)
  history_cb = tf.keras.callbacks.CSVLogger(saved_hist, separator=",", append=True)

  return [checkpointer, history_cb] # add tensorboard if needed

In [ ]:
all_X = []
all_Y = []
fulldatasetpath = './'
metadata = pd.read_csv('http://76.213.149.126:8081/metadata/UrbanSound8K.csv')
print(metadata)
for index, row in metadata.iterrows():
    
    file_name = 'fold'+str(row["fold"])+'/' + str(row["slice_file_name"])
    all_X.append(file_name.replace(".wav", "_spec.pkl"))
    all_Y.append(file_name.replace(".wav", "_y2.pkl"))

import requests
from multiprocessing.pool import ThreadPool
 
def download_url(url):
  #print("downloading: ",url)
  # assumes that the last segment after the / represents the file name
  # if url is abc/xyz/file.txt, the file name will be file.txt
  path = url.split("/")
  path = path[-2] + "/" + path[-1]
 
  r = requests.get(url, stream=True)
  if r.status_code == requests.codes.ok:
    if not os.path.exists(os.path.dirname(path)):
      try:
          os.makedirs(os.path.dirname(path))
      except OSError as exc: # Guard against race condition
          if exc.errno != errno.EEXIST:
              raise
    with open(path, 'wb') as f:
      for data in r:
        f.write(data)
  return url

def set_shapes(x, y):
    y.set_shape([4, 10])
    x.set_shape([4, 334, 224, 3])
    return x, y

#from sklearn.model_selection import train_test_split 
#x_train, x_test, y_train, y_test = train_test_split(all_X, all_Y, test_size=0.2, random_state = 42)

x_train = []
x_test = []
y_train = []
y_test = []

for x in all_X:
  if x[0:5] != "fold5":
    x_train.append(x)
    x_train.append(x.replace("_spec.pkl", "_augmented_noise_spec.pkl"))
    x_train.append(x.replace("_spec.pkl", "_augmented_pitch_1_spec.pkl"))
  else:
    x_test.append(x)


for y in all_Y:
  if y[0:5] != "fold5":
    y_train.append(y)
    y_train.append(y)
    y_train.append(y)

  else:
    y_test.append(y)
    

def file_to_arr(t: tf.Tensor):
  x = []
  y = []

  urls = []

  for a in t:
    a = np.array(a)
    # check whether we have them
    # if not, schedule download
    if not os.path.exists(a[0]):
      urls.append("http://76.213.149.126:8081/audio/" + a[0].decode('UTF-8'))
    if not os.path.exists(a[1]):
      urls.append("http://76.213.149.126:8081/audio/" + a[1].decode('UTF-8'))
  
  if (len(urls) > 0):
    ThreadPool(4).map(download_url, urls)

  for a in t:
    a = np.array(a)
    #print(a[0])
    try:
      x.append(pickle.load(open(a[0], 'rb')))
      y.append(pickle.load(open(a[1], 'rb')))
    except:
      os.remove(a[0])
      os.remove(a[1])
      return file_to_arr(t)

  x = np.asarray(x)
  x = np.swapaxes(x, 1, 2)
  #x = tf.convert_to_tensor(x)
  #x.set_shape([341, 513, 2])
  x = np.stack([x,x,x], axis=3)
  y = np.array(y).astype(int)
  res = (x,y)
  #print (res)
  return (res)

print((list(zip(x_train, y_train))))
train_ds = tf.data.Dataset.from_tensor_slices(list(zip(x_train, y_train)))
train_ds = train_ds.shuffle(len(train_ds),seed=42)
train_ds = train_ds.batch(4)
train_ds = train_ds.map((lambda x: tf.py_function(func=file_to_arr,inp=[x], Tout=(tf.float32, tf.int64))))
train_ds = train_ds.map(lambda x, y: set_shapes(x,y) )
train_ds = train_ds.prefetch(32)

test_ds = tf.data.Dataset.from_tensor_slices(list(zip(x_test, y_test)))
test_ds = test_ds.batch(4)
test_ds = test_ds.map((lambda x: tf.py_function(func=file_to_arr,inp=[x], Tout=(tf.float32, tf.int64))))
test_ds = test_ds.map(lambda x, y: set_shapes(x,y) )
test_ds = test_ds.prefetch(32)

print(test_ds.element_spec)

In [ ]:
model = create_new_model()
#model = load_model(9, 32, 2, "./drive/My Drive/wavenet-test/new2/9_32_2_skipconnections_customdropout_run2/")

In [ ]:
callbacks = create_callbacks("./")

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 334, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 11, 7, 2048)       23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                20490     
Total params: 23,608,202
Trainable params: 20,490
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
#!mkdir ./checkpoint

In [ ]:
model.fit(train_ds, validation_data=test_ds, callbacks=callbacks, epochs=300)

Epoch 1/300
5847/5847 [==============================] - ETA: 0s - loss: 8.2603 - accuracy: 0.1046
Epoch 00001: val_accuracy improved from -inf to 0.09936, saving model to ./saved_wavenet_clasifier.h5
5847/5847 [==============================] - 1588s 272ms/step - loss: 8.2603 - accuracy: 0.1046 - val_loss: 8.9889 - val_accuracy: 0.0994
Epoch 2/300
5847/5847 [==============================] - ETA: 0s - loss: 8.2603 - accuracy: 0.1046
Epoch 00002: val_accuracy did not improve from 0.09936
5847/5847 [==============================] - 829s 142ms/step - loss: 8.2603 - accuracy: 0.1046 - val_loss: 8.9889 - val_accuracy: 0.0994
Epoch 3/300
5847/5847 [==============================] - ETA: 0s - loss: 8.2602 - accuracy: 0.1046
Epoch 00003: val_accuracy did not improve from 0.09936
5847/5847 [==============================] - 1086s 186ms/step - loss: 8.2602 - accuracy: 0.1046 - val_loss: 8.9889 - val_accuracy: 0.0994
Epoch 4/300
4512/5847 [======================>.......]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-581d5aaff7cb>", line 1, in <module>
    model.fit(train_ds, validation_data=test_ds, callbacks=callbacks, epochs=300)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line 108, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py", line 1103, in fit
    callbacks.on_train_batch_end(end_step, logs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/callbacks.py", line 440, in on_train_batch_end
    self._call_batch_hook(ModeKeys.TRAIN, 'end', batch, logs=logs)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/callbacks.py", line 289, in _call_batch_hook
    self._call_batch_end_hook(mode, batc

KeyboardInterrupt: ignored

In [ ]:
!cat "./drive/My Drive/wavenet-test/new2/8_32_2_skipconnections_dropout0.1_convkernalsize2/wavenet_classifier_training_history.csv"

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer, loss=keras.losses.categorical_crossentropy, metrics=["categorical_accuracy"])

In [ ]:
[2 ** i for i in range(7)]

In [ ]:
t = iter(train_ds)

In [ ]:
g = next(t)

[array([[1.16546425e+02, 3.30120125e+01, 2.29048877e-04, ...,
        8.28410871e-03, 2.74245769e-01, 5.44586086e+00],
       [1.26469215e+02, 3.47922897e+01, 4.51437570e-03, ...,
        4.56554741e-02, 3.42672378e-01, 5.46649790e+00],
       [1.65045090e+02, 4.54606361e+01, 5.45804501e-01, ...,
        2.03821182e-01, 1.48071134e+00, 4.61989307e+00],
       ...,
       [9.38999653e-02, 4.36816514e-02, 2.26944499e-02, ...,
        1.62490236e-03, 1.41818984e-03, 2.08183168e-03],
       [8.33851248e-02, 4.29473110e-02, 3.67945954e-02, ...,
        1.16338348e-03, 1.06719648e-03, 1.20463257e-03],
       [8.52781907e-02, 2.66693570e-02, 3.95554351e-03, ...,
        3.45527253e-04, 4.02274105e-04, 2.64959992e-04]], dtype=float32), array([[1.0393705e+01, 2.8661668e+00, 2.4650497e-03, ..., 2.0881442e-03,
        7.2612691e-01, 1.4623778e+01],
       [1.1885493e+01, 3.4261134e+00, 8.0066798e-03, ..., 5.5891454e-02,
        1.0982164e+00, 1.5462135e+01],
       [1.3061634e+01, 4.2560687e+00, 

InvalidArgumentError: ignored

In [ ]:
g

In [ ]:
base_model.layers

NameError: ignored